The goal of this notebook is to show you how you can use pre-existing code you have already written in python, or how to use a functionality in Python or a Python package that is not yet available in Julia. We'll start by first running a small python code from here.

We will make use of the `PyCall.jl` package.

In [1]:
using PyCall

In [2]:
py"""1+1"""

2

The above line ran Python in the background. We can write any Python code here in the same format.

In [11]:
py"""
def py_sum2(A):
    s = 0.0
    for a in A:
        s += a
    return s
"""
sum_py = py"py_sum2"

PyObject <function py_sum2 at 0x1330420e0>

In [12]:
typeof(sum_py)

PyObject

In [14]:
v = rand(1000)
@show sum_py(v)
@show sum(v)

sum_py(v) = 490.1207953503426
sum(v) = 490.1207953503425


490.1207953503425

Now let's say you want to call some method from a previous Python code you've written in the past.
Let's say the file is `pythonsum.py`

In [15]:
python_sum = pyimport("pythonsum")

PyCall.PyError: PyError (PyImport_ImportModule

The Python package pythonsum could not be found by pyimport. Usually this means
that you did not install pythonsum in the Python version being used by PyCall.

PyCall is currently configured to use the Python version at:

/opt/anaconda3/bin/python

and you should use whatever mechanism you usually use (apt-get, pip, conda,
etcetera) to install the Python package containing the pythonsum module.

One alternative is to re-configure PyCall to use a different Python
version on your system: set ENV["PYTHON"] to the path/name of the python
executable you want to use, run Pkg.build("PyCall"), and re-launch Julia.

Another alternative is to configure PyCall to use a Julia-specific Python
distribution via the Conda.jl package (which installs a private Anaconda
Python distribution), which has the advantage that packages can be installed
and kept up-to-date via Julia.  As explained in the PyCall documentation,
set ENV["PYTHON"]="", run Pkg.build("Py

In [16]:
pushfirst!(PyVector(pyimport("sys")."path"), "") #to tell pyimport search in the local directory
python_sum = pyimport("pythonsum")

PyObject <module 'pythonsum' from '/Users/hnassar/Dropbox/juliacon2020/pythonsum.py'>

In [18]:
python_sum.py_sum(v) #PyObjectvarname.functionname

490.1207953503426

You can also import Python libraries.

In [19]:
importednumpy = pyimport("numpy")

PyObject <module 'numpy' from '/opt/anaconda3/lib/python3.7/site-packages/numpy/__init__.py'>

And now you will get all the numpy functionality...

In [20]:
importednumpy.sum(v)

490.12079535034263

There is another package that can come handy here. The `ScikitLearn` package. Let's give it a try. Just picked a random example from here: https://scikit-learn.org/stable/modules/clustering.html#mean-shift
```
>>> from sklearn.metrics.cluster import contingency_matrix
>>> x = ["a", "a", "a", "b", "b", "b"]
>>> y = [0, 0, 1, 1, 2, 2]
>>> contingency_matrix(x, y)
array([[2, 1, 0],
       [0, 1, 2]])

```

In [21]:
using ScikitLearn
@sk_import metrics.cluster : contingency_matrix

In [25]:
x = ["a", "a", "a", "b", "b", "b"]
y = [0, 0, 1, 1, 2, 2]
contingency_matrix(x, y)

2×3 Array{Int64,2}:
 2  1  0
 0  1  2

So far, we have only used simple Python code, and a very common Python package. Things might get a little more complicated when you have dependences. But we'll walk through this here. Particularly, we'll look at which version of Python Julia is using and how to make sure you have all the dependences you need.

By default, PyCall will use a Julia specific Python installation. My python version I'm currently using is here `/opt/anaconda3/bin/python`. If your Python code runs fine from whichever Python installation you are using on your machine, and you want to make Julia use it, you can do the following:

```
ENV["PYTHON"] = <path you want>
using Pkg
Pkg.build("PyCall")
```

Again, if your code runs under this python installation it should now run from within Julia. Now let's say want to stick with the Julia specific installation.

In [32]:
# will now move to the terminal
# ENV["PYTHON"] = ""
# ]build PyCall
# pyimport_conda("igraph","python-igraph","conda-forge")

""

In [36]:
?pyimport_conda

search: pyimport_conda



```
pyimport_conda(modulename, condapkg, [channel])
```

Returns the result of `pyimport(modulename)` if possible.   If the module is not found, and PyCall is configured to use the Conda Python distro (via the Julia Conda package), then automatically install `condapkg` via `Conda.add(condapkg)` and then re-try the `pyimport`.   Other Anaconda-based Python installations are also supported as long as their `conda` program is functioning.

If PyCall is not using Conda and the `pyimport` fails, throws an exception with an error message telling the user how to configure PyCall to use Conda for automated installation of the module.

The third argument, `channel` is an optional Anaconda "channel" to use for installing the package; this is useful for packages that are not included in the default Anaconda package listing.


To recap, you have control over which Python version to allow PyCall to use. If you use the Julia specific version, you get the benefit of being able to easily install packages via `pyimport_coda`. If you choose to use your own Python specific installation, make sure everything runs from Python, and you can change the location of what Python version to call from Julia via `ENV["PYTHON"] = xxx`.